In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
spark = SparkSession.builder.appName("IrisML").getOrCreate()

25/11/24 17:08:21 WARN Utils: Your hostname, spark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/11/24 17:08:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 17:08:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/24 17:08:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/24 17:08:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
file_path = "/home/spark/Downloads/archive/Iris.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
df.show(150)

+---+-------------+------------+-------------+------------+---------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|        Species|
+---+-------------+------------+-------------+------------+---------------+
|  1|          5.1|         3.5|          1.4|         0.2|    Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|    Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|    Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|    Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|    Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|    Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|    Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|    Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|    Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|    Iris-setosa|
| 11|       

In [5]:
from pyspark.ml.feature import StringIndexer

In [6]:
indexer = StringIndexer(inputCol="Species", outputCol="label")
df = indexer.fit(df).transform(df)

In [7]:
df = df.drop("features")

In [8]:
feature_cols = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

In [9]:
df = df.select("features", "label")

In [10]:
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)

In [11]:
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")

In [12]:
model = dt.fit(train_data)

In [13]:
predictions = model.transform(test_data)

In [16]:
predictions.select("features", "label", "prediction").show(150)

+-----------------+-----+----------+
|         features|label|prediction|
+-----------------+-----+----------+
|[4.4,3.0,1.3,0.2]|  0.0|       0.0|
|[4.6,3.2,1.4,0.2]|  0.0|       0.0|
|[4.6,3.6,1.0,0.2]|  0.0|       0.0|
|[4.7,3.2,1.3,0.2]|  0.0|       0.0|
|[4.8,3.1,1.6,0.2]|  0.0|       0.0|
|[4.8,3.4,1.6,0.2]|  0.0|       0.0|
|[4.8,3.4,1.9,0.2]|  0.0|       0.0|
|[4.9,3.1,1.5,0.1]|  0.0|       0.0|
|[4.9,3.1,1.5,0.1]|  0.0|       0.0|
|[5.0,2.3,3.3,1.0]|  1.0|       1.0|
|[5.0,3.0,1.6,0.2]|  0.0|       0.0|
|[5.0,3.4,1.6,0.4]|  0.0|       0.0|
|[5.0,3.5,1.3,0.3]|  0.0|       0.0|
|[5.0,3.5,1.6,0.6]|  0.0|       0.0|
|[5.1,2.5,3.0,1.1]|  1.0|       1.0|
|[5.1,3.4,1.5,0.2]|  0.0|       0.0|
|[5.1,3.5,1.4,0.2]|  0.0|       0.0|
|[5.1,3.8,1.6,0.2]|  0.0|       0.0|
|[5.2,3.4,1.4,0.2]|  0.0|       0.0|
|[5.2,3.5,1.5,0.2]|  0.0|       0.0|
|[5.3,3.7,1.5,0.2]|  0.0|       0.0|
|[5.4,3.0,4.5,1.5]|  1.0|       1.0|
|[5.4,3.4,1.5,0.4]|  0.0|       0.0|
|[5.4,3.7,1.5,0.2]|  0.0|       0.0|
|

In [15]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print(f"Decision Tree Accuracy: {accuracy*100:.2f}%")

Decision Tree Accuracy: 97.83%
